In [1]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2

In [2]:
include("../../../util2.j")

colnorm (generic function with 1 method)

In [3]:
# unnecessary packages #

#using Pkg
#Pkg.add("UnicodePlots")
using UnicodePlots   # check the structure of the sparse matrix
using BenchmarkTools

using StatsPlots
using MCMCChains
using PrettyTables

In [4]:
#using Pkg
#Pkg.add("ProgressMeter");

In [5]:
# Set the parameters for SLMC model #

N = 1200 # sample size
N1 = 1000; N2 = 1000;
q = 2; p = 2; K = 2
Σ = [0.3 0.0 
     0.0 0.2];
β = [1.0 -1.0
     -5.0 2.0];
ϕ1 = 6.0; ϕ2 = 6.0; ν1 = 0.5; ν2 = 0.5;   # parameter for the independent F
Λ = [1.0 -1.0
     0.0 2.0]; # loading matrix


In [6]:
# Generate simulation data #

Random.seed!(1234);
coords = rand(N, 2);                                          # random location over unit square
X = hcat(fill(1, (N,)), rand(N));                             # design matrix
D = pairwise(Euclidean(), coords, dims = 1);                  # distance matrix
ρ1 = exp.(-ϕ1 * D); ρ2 = exp.(-ϕ2 * D);    # covariance matrix
ω = [rand(MvNormal(ρ1), 1) rand(MvNormal(ρ2), 1)] * Λ; # latent process
Y = X * β + ω + transpose(rand(MvNormal(Σ), N));              # response matrix

In [7]:
# Some data preparations #

ordx = 1:N;                               # sort order based on the first coordinates
X_ord = X[ordx, :]; Y_ord = Y[ordx, :]; ω_ord = ω[ordx, :];   # sorted data
ω_incp_obs = ω_ord + fill(1.0, (N, 1)) * transpose(β[1, :]); # latent process + intercept
coords_ord = coords[ordx, :];
S1_ind = sample(1:N, N1, replace = false, ordered = true);    # observed location index for 1st response
S2_ind = sample(1:N, N2, replace = false, ordered = true);    # observed location index for 2nd response
S = sort(union(S1_ind, S2_ind));                              # observed index set

In [8]:
# check the plot of the data 
using RCall
@rput ω_incp_obs
@rput coords_ord
@rput S
R"""
library(MBA)
library(classInt)
library(RColorBrewer)
library(sp)
library(coda)
library(spBayes)
library(fields)

h <- 12
surf.raw1 <- mba.surf(cbind(coords_ord[S, ], ω_incp_obs[S, 1]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.raw2 <- mba.surf(cbind(coords_ord[S, ], ω_incp_obs[S, 2]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.brks <- classIntervals(c(surf.raw1[["z"]], surf.raw2[["z"]]), 500, 'pretty')$brks
col.pal <- colorRampPalette(brewer.pal(11,'RdBu')[11:1])
xlim <- c(0, 1.13)

zlim <- range(c(surf.raw1[["z"]], surf.raw2[["z"]]))

# size for the mapping of w               
width <- 360
height <- 360
pointsize <- 16

png(paste("../../pics/sim4_map-w1_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.raw1)
plot(coords_ord[S, ], typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../../pics/sim4_map-w2_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.raw2)
plot(coords_ord[S, ], typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

"""

┌ Warning: Package RCall does not have AxisArrays in its dependencies:
│ - If you have RCall checked out for development and have
│   added AxisArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with RCall
│ Loading AxisArrays into RCall from project dependency, future warnings for RCall are suppressed.
└ @ nothing nothing:840
┌ Warning: RCall.jl: Loading required package: magic
│ Loading required package: abind
│ Loading required package: Formula
│ Loading required package: Matrix
└ @ RCall /home/lu/.julia/packages/RCall/ffM0W/src/io.jl:113
┌ Warning: RCall.jl: Loading required package: spam
│ Loading required package: dotCall64
│ Loading required package: grid
│ Spam version 2.2-2 (2019-03-07) is loaded.
│ Type 'help( Spam)' or 'demo( spam)' for a short introduction 
│ and overview of this package.
│ Help for individual functions is also obtained by adding the
│ suffix '.spam' to 

RObject{IntSxp}
null device 
          1 


In [9]:
@save "sim2data.jld"